In [0]:
%run ./00_Setup_Config

We filter the US Accidents dataset to focus on dense urban environments, specifically targeting larger cities like Boston. We use Boston as our primary evaluation set to simulate how the model might perform in a similar metropolitan climate like Montreal, focusing on how urban infrastructure (signals, crossings) interacts with weather to impact accident severity.

In [0]:
%sql
SELECT CONCAT(City, ', ', State) AS Loc, COUNT(CONCAT(City, ', ', State)) AS accident_volume, ROUND(AVG(Severity), 2) AS avg_severity
FROM dblearning.us_accidents.bronze_accidents
WHERE City = 'Boston' AND State = 'MA'
GROUP BY CONCAT(City, ', ', State)
ORDER BY accident_volume;

In [0]:
%sql
CREATE OR REPLACE TABLE dblearning.us_accidents.silver_accidents
AS
-- 1. Identify "Urban" cities based on volume
WITH city_counts AS (
  SELECT City, COUNT(*) as accident_volume
  FROM dblearning.us_accidents.bronze_accidents
  GROUP BY City
  HAVING accident_volume >= 1000
),

-- 2. Extract time features and filter
time_features AS (
  SELECT b.*,
    TIMESTAMPDIFF(SECOND, b.Start_Time, b.End_Time) / 60 AS Duration_Minutes,
    HOUR(b.Start_Time) AS Start_Hour,
    DAYOFWEEK(b.Start_Time) AS Start_DOW,
    MONTH(b.Start_Time) AS Start_Month,
    YEAR(b.Start_Time) AS Start_Year
  FROM dblearning.us_accidents.bronze_accidents b
  INNER JOIN city_counts c ON b.City = c.City
  WHERE b.Start_Time IS NOT NULL 
    AND b.End_Time IS NOT NULL
    AND b.Start_Lat IS NOT NULL 
    AND b.Start_Lng IS NOT NULL
    AND b.City IS NOT NULL
    AND b.Zipcode IS NOT NULL
    AND b.Street IS NOT NULL
),

-- 3. Calculate Weather Mode (Most frequent condition per County/Month)
-- We do this separately to avoid the aggregate-window-function error
weather_stats AS (
  SELECT 
    County, 
    Start_Month, 
    Weather_Condition,
    COUNT(*) as condition_freq,
    ROW_NUMBER() OVER(PARTITION BY County, Start_Month ORDER BY COUNT(*) DESC) as rank
  FROM time_features
  WHERE Weather_Condition IS NOT NULL
  GROUP BY County, Start_Month, Weather_Condition
),

weather_modes AS (
  SELECT County, Start_Month, Weather_Condition as mode_weather
  FROM weather_stats
  WHERE rank = 1
)

-- 4. Final selection joining with weather modes
SELECT
  t.ID,
  t.Severity,
  t.Distance_mi AS Distance_Mi,
  t.Start_Lat,
  t.Start_Lng,
  t.Street,
  t.City,
  t.County,
  t.State,
  t.Zipcode,
  COALESCE(t.Temperature_F, AVG(t.Temperature_F) OVER(PARTITION BY t.State, t.Start_Month)) as Temperature_F,
  COALESCE(t.Precipitation_in, 0.0) AS Precipitation_In,
  COALESCE(t.Visibility_mi, PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY t.Visibility_mi) OVER()) as Visibility_Mi,
  -- Impute Weather: Use the calculated mode from our mode CTE
  COALESCE(t.Weather_Condition, m.mode_weather, 'Unknown') AS Weather_Condition,
  t.Amenity, t.Bump, t.Crossing, t.Give_Way, t.Junction, t.No_Exit, t.Railway,
  t.Roundabout, t.Station, t.Stop, t.Traffic_Calming, t.Traffic_Signal, t.Turning_Loop,
  CASE 
    WHEN t.Sunrise_Sunset IS NOT NULL THEN t.Sunrise_Sunset
    WHEN t.Start_Hour BETWEEN 6 AND 18 THEN 'Day'
    ELSE 'Night'
  END AS Sunrise_Sunset,
  t.Start_Hour, t.Start_DOW, t.Start_Month, t.Start_Year, t.Duration_Minutes
FROM time_features t
LEFT JOIN weather_modes m ON t.County = m.County AND t.Start_Month = m.Start_Month;

In [0]:
%sql
SELECT COUNT(*) AS num_rows FROM dblearning.us_accidents.silver_accidents;